# PlayHQ Fixture Scraping

This system allows to scrape game fixtures from [PlayHQ](http://playhq.com/
) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs). 

It will produce a CSV file ready to be uploaded as Schedule in [TeamApp](https://brunswickmagicbasketball.teamapp.com/).

The *Public* APIs only require a header parameters to get a successful response, which includes `x-api-key` (also referred to as the Client ID) and `x-phq-tenant` (refers to the sport/association - in this case `bv`). Note that the Private APIs are not available to clubs and associations.

Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

Contact: Sebastian Sardina (sssardina@gmail.com)

In [14]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re
import os
import calendar, datetime

import playhq_api as phq
from config import *

# import dtale

## 1. Set-up application

First, creating a connection to the PlayHQ Public API. Remember to set-up file `config.py` for club and season configuration.


The *Public* APIs only require the below header parameters to get a successful response:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.

The feature to create new API credentials is disabled for you and can only be actioned by a Super Administrator role within the Play HQ portal. Please use the credentials provided to call the PlayHQ public APIs.

In [15]:
GAME_DATE = phq.next_day(calendar.SATURDAY) # get the date of the upcoming Saturday (game day in competition)
GAME_DATE_TIMESTAMP = pd.to_datetime(GAME_DATE).tz_localize(TIMEZONE)
GAME_DATE_NAME = GAME_DATE_TIMESTAMP.strftime("%A %B %d, %Y (%Y/%m/%d)") # Saturday August 06, 2022

phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE)
print(f"Set-up games for club {CLUB_NAME} for upcoming Saturday is: {GAME_DATE_NAME}")

PLAYHQ_URL=f"https://bv.playhq.com/org/{ORG_ID}/games?date={GAME_DATE_TIMESTAMP.strftime('%Y-%m-%d')}"
print("Game day at PlayHQ:", PLAYHQ_URL)
print("Club PlayHQ link: ", PLAYHQ_CLUB_SEASON)

Set-up games for club Brunswick Magic Basketball Club for upcoming Saturday is: Saturday September 10, 2022 (2022/09/10)
Game day at PlayHQ: https://bv.playhq.com/org/8c4d5431-eaa5-4644-82ac-992abe224b88/games?date=2022-09-10
Club PlayHQ link:  https://bit.ly/bmbc-w22


Next check if PlayHQ x-api-key is available already, otherwise ask user for it to access PlayHQ API.

In [ ]:
if X_API_KEY is None:
  X_API_KEY = input("Enter your x-api-key:")
else:
  print("x-api-key is already defined!")

Now get the teams of the club; it will be used below.

In [16]:
season_id = phq_club.get_season_id(SEASON)
teams_df = phq_club.get_season_teams(season_id)
teams_df

,id,name,grade.id,grade.name,grade.url,age
0,d7486008-b2fe-47db-8011-01265eaf1cfe,Magic U8 Mixed Purple,be7076cb-a1de-4033-8082-b52c8c149861,Saturday U8 Mixed Division 1/2,https://www.playhq.com/basketball-victoria/org...,8
1,8d635867-0718-4f70-a1b8-1a224992d294,Magic U16 Girls Gold,19b86372-2ad5-4a5a-9592-467f826f7d63,Saturday U16 Girls Division 1/2,https://www.playhq.com/basketball-victoria/org...,16
2,69d69214-641d-4ece-a2ce-ca9534865553,Magic U16 Boys Purple,be4b2b9c-0bc1-44e2-ac3a-b89bb10230f8,Saturday U16 Boys Division 3,https://www.playhq.com/basketball-victoria/org...,16
3,135c3bab-3004-4426-b9c6-3669b48ce271,Magic U16 Boys Gold,ceac1ec4-c86b-47c6-9e67-124d923ea713,Saturday U16 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,16
4,99f369ed-5c63-40e7-8591-8cd18fba7ec0,Magic U14 Boys Gold,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
5,8b2bc68b-9fda-435c-9c13-297d1d3187d1,Magic U14 Girls Purple,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
6,9ebeb141-4023-4744-84f1-f58957b966b2,Magic U14 Girls Gold,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
7,2b844e90-6575-47bc-bb49-f55840f8e6f8,Magic U14 Girls Black,237a9d9a-52c5-4136-b179-c01b10ad93ae,Saturday U14 Girls Division 4,https://www.playhq.com/basketball-victoria/org...,14
8,623c00bb-302b-4492-a988-90aacc3e7768,Magic U14 Boys Purple,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
9,a1e571f6-8ac4-49d2-9787-f62b35947eca,Magic U14 Boys Black,b877a73b-037d-460b-acb5-a2c45e5debee,Saturday U14 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,14


## 2. Get the upcoming games for those teams

Using the teams of the club, extract all upcoming games for the Club's teams.

In [17]:
upcoming_games_df = phq_club.get_games(teams_df, GAME_DATE_TIMESTAMP)

if upcoming_games_df is not None:
    print(f'There were {upcoming_games_df.shape[0]} games extracted for game day: {GAME_DATE_NAME}')
    upcoming_games_df[phq.GAMES_COLS]
else:
    print("No games for date: ", GAME_DATE_NAME)

2022-09-08 07:25:22 INFO No games for team: Magic U8 Mixed Purple
2022-09-08 07:25:22 INFO Games extracted for team: Magic U16 Girls Gold
2022-09-08 07:25:22 INFO No games for team: Magic U16 Boys Purple
2022-09-08 07:25:23 INFO No games for team: Magic U16 Boys Gold
2022-09-08 07:25:23 INFO No games for team: Magic U14 Boys Gold
2022-09-08 07:25:24 INFO No games for team: Magic U14 Girls Purple
2022-09-08 07:25:24 INFO Games extracted for team: Magic U14 Girls Gold
2022-09-08 07:25:24 INFO Games extracted for team: Magic U14 Girls Black
2022-09-08 07:25:25 INFO Games extracted for team: Magic U14 Boys Purple
2022-09-08 07:25:25 INFO No games for team: Magic U14 Boys Black
2022-09-08 07:25:26 INFO No games for team: Magic U12 Girls Gold
2022-09-08 07:25:26 INFO Games extracted for team: Magic U12 Girls Black
2022-09-08 07:25:26 INFO No games for team: Magic U12 Boys Purple
2022-09-08 07:25:27 INFO No games for team: Magic U12 Boys Gold
2022-09-08 07:25:27 INFO No games for team: Magic 

There were 10 games extracted for game day: Saturday September 10, 2022 (2022/09/10)


## 3. Convert to TeamApp CSV format

Next, we convert tthe PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

In [18]:
games_tapps_df = phq.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP)
print("Done computing the games for Teams App")
games_tapps_df.sample(3)

Done computing the games for Teams App


,team_name,team_id,round_name,round_abbreviatedName,event_name,opponent,schedule_timestamp,start_date,end_date,start_time,...,attendance_tracking,duty_roster,ticketing,reference_id,venue,court,geo,game_url,grade_url,description
6,U10 Boys Purple,55b0eb0f-f3e2-4566-bcde-657b1b872103,Round 18,R18,U10 Boys Purple - Round 18,Warriors U10 Boys Black,2022-09-10 09:30:00+10:00,2022-09-10,2022-09-10,09:30:00,...,0,1,0,,St John's College (Preston),Court 1,"(-37.74845,144.99944)",https://tinyurl.com/2m25q8lq,https://tinyurl.com/293nugdx,RSVP is YES by default - if you cannot make it...
8,U10 Girls Gold,728b7218-1f46-41be-a7c7-9d8d82376c89,Round 18,R18,U10 Girls Gold - Round 18,Warriors U10 Girls,2022-09-10 09:30:00+10:00,2022-09-10,2022-09-10,09:30:00,...,0,1,0,,Mercy College,Court 1,"(-37.72981,144.96403)",https://tinyurl.com/2ogpe6qv,https://tinyurl.com/2ybq5jez,RSVP is YES by default - if you cannot make it...
5,U12 Boys Black,b5627a3f-0010-4c11-a658-896f7cc5127a,Finals Round 2,FR2,U12 Boys Black - Finals Round 2,St Fidelis U12 Boys Gold,2022-09-10 09:30:00+10:00,2022-09-10,2022-09-10,09:30:00,...,0,1,0,,Oak Park Stadium,Court 1,"(-37.713213,144.925617)",https://tinyurl.com/2qz384jw,https://tinyurl.com/2yyuqxnd,RSVP is YES by default - if you cannot make it...


Next filter and re-order columns relevant to Teams App schedule format.

In [19]:
TAPP_COLS_CSV = ['event_name', 'team_name', 'start_date', 'end_date', 'start_time', 'end_time', 'description', 'venue', 'location', 'access_groups', 'rsvp', 'comments', 'attendance_tracking', 'duty_roster', 'ticketing']

team_apps_csv_df = games_tapps_df.loc[:, TAPP_COLS_CSV]
team_apps_csv_df.sample(4)


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing
6,U10 Boys Purple - Round 18,U10 Boys Purple,2022-09-10,2022-09-10,09:30:00,10:15:00,RSVP is YES by default - if you cannot make it...,St John's College (Preston),"21 Railway Place West, Preston",U10 Boys Purple,1,1,0,1,0
4,U12 Girls Black - Finals Round 2,U12 Girls Black,2022-09-10,2022-09-10,10:30:00,11:15:00,RSVP is YES by default - if you cannot make it...,Mercy College,"760 Sydney Road, Coburg",U12 Girls Black,1,1,0,1,0
8,U10 Girls Gold - Round 18,U10 Girls Gold,2022-09-10,2022-09-10,09:30:00,10:15:00,RSVP is YES by default - if you cannot make it...,Mercy College,"760 Sydney Road, Coburg",U10 Girls Gold,1,1,0,1,0
2,U14 Girls Black - Finals Round 2,U14 Girls Black,2022-09-10,2022-09-10,11:30:00,12:15:00,RSVP is YES by default - if you cannot make it...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U14 Girls Black,1,1,0,1,0


In [20]:
# Inspect description of one record
print(team_apps_csv_df.iloc[4]['event_name'])
print(team_apps_csv_df.iloc[4]['description'])

U12 Girls Black - Finals Round 2
RSVP is YES by default - if you cannot make it, please let your Team Manager know as soon as possible.
Opponent: Jets U12 Girls Yellow
Venue: Mercy College (Court 1)
Address: 760 Sydney Road, Coburg 
Google Maps coord: https://maps.google.com/?q=(-37.72981,144.96403)

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- 45 min schedule with 18 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: https://tinyurl.com/2en467uq
Check the round in PlayHQ: https://tinyurl.com/23shhpw4
All clubs in PlayHQ: https://bit.ly/bmbc-w22



### Extract and add BYE games

In [21]:
# Extract the date of the round
date = team_apps_csv_df.iloc[1]['start_date']
date = GAME_DATE

print(f"Extract BYE games for games on {GAME_DATE_NAME}")

# Extract teams that do not have a game
playing_teams = games_tapps_df['team_id'].tolist()
bye_teams_df = teams_df.loc[~teams_df['id'].isin(playing_teams)].copy()

# Add entries for BYE teams
if not bye_teams_df.empty:  # there are BYE games
    print("There are BYE games...")

    bye_teams_df = bye_teams_df[['id', 'name']]

    bye_teams_df['team_name'] = bye_teams_df.apply(lambda x: re.search("U.*", x['name']).group(0), axis=1)
    bye_teams_df['access_groups'] = bye_teams_df['team_name']
    bye_teams_df['event_name'] = bye_teams_df['team_name'] + " - BYE"
    bye_teams_df['start_date'] = date
    bye_teams_df['end_date'] = date
    bye_teams_df['start_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['end_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['description'] = DESC_BYE_TAPP
    bye_teams_df['location'] = ""
    bye_teams_df['venue'] = "BYE"

    bye_teams_df['rsvp'] = 0
    bye_teams_df['comments'] = 0
    bye_teams_df['attendance_tracking'] = 0
    bye_teams_df['duty_roster'] = 0
    bye_teams_df['ticketing'] = 0
    bye_teams_df['reference_id'] = ""

    bye_teams_df = bye_teams_df[TAPP_COLS_CSV]
    team_apps_csv_df = pd.concat([team_apps_csv_df, bye_teams_df])
    team_apps_csv_df.drop_duplicates(inplace=True)


if bye_teams_df.size > 0:
    for x in bye_teams_df['team_name'].values:
        print(x)
else:
    print("No BYE games this round...")

bye_teams_df

Extract BYE games for games on Saturday September 10, 2022 (2022/09/10)
There are BYE games...
U8 Mixed Purple
U16 Boys Purple
U16 Boys Gold
U14 Boys Gold
U14 Girls Purple
U14 Boys Black
U12 Girls Gold
U12 Boys Purple
U12 Boys Gold
U12 Boys Diamond


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing
0,U8 Mixed Purple - BYE,U8 Mixed Purple,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U8 Mixed Purple,0,0,0,0,0
2,U16 Boys Purple - BYE,U16 Boys Purple,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U16 Boys Purple,0,0,0,0,0
3,U16 Boys Gold - BYE,U16 Boys Gold,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U16 Boys Gold,0,0,0,0,0
4,U14 Boys Gold - BYE,U14 Boys Gold,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Boys Gold,0,0,0,0,0
5,U14 Girls Purple - BYE,U14 Girls Purple,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Girls Purple,0,0,0,0,0
9,U14 Boys Black - BYE,U14 Boys Black,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Boys Black,0,0,0,0,0
10,U12 Girls Gold - BYE,U12 Girls Gold,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Gold,0,0,0,0,0
12,U12 Boys Purple - BYE,U12 Boys Purple,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Boys Purple,0,0,0,0,0
13,U12 Boys Gold - BYE,U12 Boys Gold,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Boys Gold,0,0,0,0,0
14,U12 Boys Diamond - BYE,U12 Boys Diamond,2022-09-10,2022-09-10,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Boys Diamond,0,0,0,0,0


## 5. Save to CSV file for Teams App import

### 5.1. FINAL CHECK

Finally, report the games to be written into Schedule CSV file and **CHECK ALL IS GOOD!**

Particularly, look for games that are schedule but **PENDING** and without all details (time or venue).

In [22]:
team_apps_csv_df.columns
team_apps_csv_df[['team_name', 'start_date', 'start_time', 'venue']]

,team_name,start_date,start_time,venue
0,U16 Girls Gold,2022-09-10,13:30:00,Coburg Basketball Stadium
1,U14 Girls Gold,2022-09-10,12:30:00,Pascoe Vale Girls College
2,U14 Girls Black,2022-09-10,11:30:00,Coburg Basketball Stadium
3,U14 Boys Purple,2022-09-10,12:30:00,Mercy College
4,U12 Girls Black,2022-09-10,10:30:00,Mercy College
5,U12 Boys Black,2022-09-10,09:30:00,Oak Park Stadium
6,U10 Boys Purple,2022-09-10,09:30:00,St John's College (Preston)
7,U10 Girls Purple,2022-09-10,08:30:00,Coburg Basketball Stadium
8,U10 Girls Gold,2022-09-10,09:30:00,Mercy College
9,U10 Boys Gold,2022-09-10,09:30:00,Coburg Senior High School


### 5.2. Write a TeamAPP Schedule CSV

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [24]:
file_csv = os.path.join(OUTPUT_PATH, f"schedule-teamsapp-{date.strftime('%Y_%m_%d')}.csv")

print(f'Saving TeamAPP schedule CSV file for games on game date: {GAME_DATE_NAME}')
print('File to save TeamApp schedule:', file_csv)
team_apps_csv_df.to_csv(file_csv, index=False)

Saving TeamAPP schedule CSV file for games on game date: Saturday September 10, 2022 (2022/09/10)
File to save TeamApp schedule: Brunswick_Magics/2022.01.Winter/fixture/schedule-teamsapp-2022_09_10.csv


### 5.3. Write fixture dataframe too



In [25]:
import datetime
import os

now = datetime.datetime.now() # current date and time

now_str = now.strftime("%Y-%m-%d_%H:%M:%S")
upcoming_games_df.to_pickle(os.path.join(OUTPUT_PATH, f"upcoming_games_df-{now_str}.pkl"))
team_apps_csv_df.to_pickle(os.path.join(OUTPUT_PATH, f"team_apps_csv_df-{now_str}.pkl"))

print(f"Finished saving dataframes: {now.strftime('%d/%m/%Y, %H:%M:%S')}")

Finished saving dataframes: 08/09/2022, 07:26:58


# ------------ END FIXTURE PUBLISHING ------------

## 6. Re-check Fixture

On Friday, just before the game day, re-check to see if any game has been changed (e.g., venue, time).

First, re-extract games in another dataframe:

In [8]:
# Recover pikle saved dataframe (comment if using above directly)
FILE = "upcoming_games_df-2022-09-06_17:14:47.pkl"
upcoming_games_df = pd.read_pickle(os.path.join(OUTPUT_PATH, FILE))

upcoming_games_df[phq.GAMES_COLS]

,team_name,status,schedule_timestamp,venue_name
0,U16 Girls Gold,UPCOMING,2022-09-10 13:30:00+10:00,Coburg Basketball Stadium
1,U14 Girls Black,UPCOMING,2022-09-10 11:30:00+10:00,Coburg Basketball Stadium
2,U14 Boys Purple,UPCOMING,2022-09-10 12:30:00+10:00,Mercy College
3,U12 Girls Black,UPCOMING,2022-09-10 10:30:00+10:00,Mercy College
4,U12 Boys Black,UPCOMING,2022-09-10 09:30:00+10:00,Oak Park Stadium


In [7]:
upcoming_games2_df = phq_club.get_games(teams_df, GAME_DATE_TIMESTAMP)

print(f'There were {upcoming_games2_df.shape[0]} games extracted for game day: {GAME_DATE_NAME}')
upcoming_games2_df[phq.GAMES_COLS]

2022-09-07 15:57:59 INFO No games for team: Magic U8 Mixed Purple
2022-09-07 15:57:59 INFO Games extracted for team: Magic U16 Girls Gold
2022-09-07 15:57:59 INFO No games for team: Magic U16 Boys Purple
2022-09-07 15:57:59 INFO No games for team: Magic U16 Boys Gold
2022-09-07 15:58:00 INFO No games for team: Magic U14 Boys Gold
2022-09-07 15:58:00 INFO No games for team: Magic U14 Girls Purple
2022-09-07 15:58:00 INFO No games for team: Magic U14 Girls Gold
2022-09-07 15:58:01 INFO Games extracted for team: Magic U14 Girls Black
2022-09-07 15:58:01 INFO Games extracted for team: Magic U14 Boys Purple
2022-09-07 15:58:01 INFO No games for team: Magic U14 Boys Black
2022-09-07 15:58:01 INFO No games for team: Magic U12 Girls Gold
2022-09-07 15:58:02 INFO Games extracted for team: Magic U12 Girls Black
2022-09-07 15:58:02 INFO No games for team: Magic U12 Boys Purple
2022-09-07 15:58:02 INFO No games for team: Magic U12 Boys Gold
2022-09-07 15:58:03 INFO No games for team: Magic U12 Boy

There were 9 games extracted for game day: Saturday September 10, 2022 (2022/09/10)


,team_name,status,schedule_timestamp,venue_name
0,U16 Girls Gold,UPCOMING,2022-09-10 13:30:00+10:00,Coburg Basketball Stadium
1,U14 Girls Black,UPCOMING,2022-09-10 11:30:00+10:00,Coburg Basketball Stadium
2,U14 Boys Purple,UPCOMING,2022-09-10 12:30:00+10:00,Mercy College
3,U12 Girls Black,UPCOMING,2022-09-10 10:30:00+10:00,Mercy College
4,U12 Boys Black,UPCOMING,2022-09-10 09:30:00+10:00,Oak Park Stadium
5,U10 Boys Purple,UPCOMING,2022-09-10 09:30:00+10:00,St John's College (Preston)
6,U10 Girls Purple,UPCOMING,2022-09-10 08:30:00+10:00,Coburg Basketball Stadium
7,U10 Girls Gold,UPCOMING,2022-09-10 09:30:00+10:00,Mercy College
8,U10 Boys Gold,UPCOMING,2022-09-10 09:30:00+10:00,Coburg Senior High School


Now, check for differences if any:

In [12]:
cols = ['team_name', 'schedule_timestamp', 'venue_name']

print("Report games that have changed since last extraction:")
teams_changed = pd.concat([upcoming_games2_df[cols], upcoming_games_df[cols]]).drop_duplicates(keep=False)['team_name'].unique()

changes_df = upcoming_games_df[cols].query("team_name in @teams_changed")
changes_df.merge(upcoming_games2_df[cols], how="outer", on="team_name", suffixes=('_new', '_old'))

teams_changed

Report games that have changed since last extraction:


array(['U10 Boys Purple', 'U10 Girls Purple', 'U10 Girls Gold',
       'U10 Boys Gold'], dtype=object)

In [14]:
upcoming_games_df

,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp,venue
0,642b122f-8363-4b97-a671-2bc3fea673b7,U16 Girls Gold,8d635867-0718-4f70-a1b8-1a224992d294,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-09-06 14:50:37+10:00,2022-09-06 14:50:37+10:00,None,[{'id': '7fcdc0ba-d6ce-498a-9091-f1012149e177'...,19b86372-2ad5-4a5a-9592-467f826f7d63,...,Crt4,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2022-09-10 13:30:00+10:00,NaN
1,11455cc7-6604-4c77-a67a-2294adf328c7,U14 Girls Black,2b844e90-6575-47bc-bb49-f55840f8e6f8,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-09-06 13:46:34+10:00,2022-09-06 13:46:34+10:00,None,[{'id': '7c448a45-0453-46de-be3b-58da83a397fc'...,237a9d9a-52c5-4136-b179-c01b10ad93ae,...,Crt3,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2022-09-10 11:30:00+10:00,NaN
2,12b83058-82d0-4152-83d6-d22875c6a9c0,U14 Boys Purple,623c00bb-302b-4492-a988-90aacc3e7768,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-09-06 13:04:46+10:00,2022-09-06 13:04:46+10:00,None,[{'id': '6cb5d768-f210-4790-b215-cb68863bf68a'...,d900616b-0f52-4d3e-8090-4eaf877491bb,...,Crt1,760 Sydney Road,3058,Coburg,VIC,Australia,-37.729810,144.964030,2022-09-10 12:30:00+10:00,NaN
3,2b760033-894d-4d45-b48c-25db02902a45,U12 Girls Black,a38bece0-6357-4ca6-9415-096fb9f3a236,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-09-06 12:37:55+10:00,2022-09-06 12:37:56+10:00,None,[{'id': '775ef041-5dd4-4a97-b26a-84592cf3c4b0'...,6c66df7f-4c4d-49a1-bf17-7fca91a3aa60,...,Crt1,760 Sydney Road,3058,Coburg,VIC,Australia,-37.729810,144.964030,2022-09-10 10:30:00+10:00,NaN
4,80cdbfed-b89e-42e1-a519-36fcf3bc7aea,U12 Boys Black,b5627a3f-0010-4c11-a658-896f7cc5127a,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-09-05 18:19:13+10:00,2022-09-06 12:31:18+10:00,None,[{'id': 'b5627a3f-0010-4c11-a658-896f7cc5127a'...,f02a6b6a-ab3f-4063-bf13-5ce969bc7b9c,...,Crt1,9 Hillcrest Road,3046,Oak Park,VIC,Australia,-37.713213,144.925617,2022-09-10 09:30:00+10:00,NaN


In [26]:
upcoming_games_df[cols].merge(upcoming_games2_df[cols], indicator = True, how='outer', on="team_name", suffixes=('_new', '_old')).loc[lambda x : x['_merge']!='both']


,team_name,schedule_timestamp_new,venue_name_new,schedule_timestamp_old,venue_name_old,_merge
5,U10 Boys Purple,NaT,NaN,2022-09-10 09:30:00+10:00,St John's College (Preston),right_only
6,U10 Girls Purple,NaT,NaN,2022-09-10 08:30:00+10:00,Coburg Basketball Stadium,right_only
7,U10 Girls Gold,NaT,NaN,2022-09-10 09:30:00+10:00,Mercy College,right_only
8,U10 Boys Gold,NaT,NaN,2022-09-10 09:30:00+10:00,Coburg Senior High School,right_only


### Check a particular team

In [ ]:
upcoming_games_df.query("team_name == 'U14 Girls Black'")